First, let's load all the necessary libraries:

In [1]:
library(openxlsx)
library(ggplot2)
library(ggrepel)

Warning message:
“package ‘openxlsx’ was built under R version 3.4.4”Warning message:
“package ‘ggplot2’ was built under R version 3.4.4”Warning message:
“package ‘ggrepel’ was built under R version 3.4.4”

Loading the functions:

In [3]:
source('functions.R')
source("visualizations.R")

Next we'll read the peak information and intensities from the provided Excel file:

In [4]:
X <- read.xlsx("HILIC_NEG_TEST.xlsx", sheet = 1)
P <- read.xlsx("HILIC_NEG_TEST.xlsx", sheet = 2)

# compute the median peak area for each feature, used in visualizations
P$MPA <- sapply(X[P$Name], median, na.rm = TRUE)

ERROR: Error in read.xlsx.default("HILIC_NEG_TEST.xlsx", sheet = 1): File does not exist.


This function finds connections between features according to set conditions: correlation threshhold and retention time window. If two features have a Pearson correlation coefficient above the set threshold and their retention times differ less than the set window, they are deemed connected.

The function returns a data frame with one row per connection, with the following columns:
- x & y: the names of the features
- cor: the pearson correlation coefficient
- mz_diff: the mass difference between the features
- rt_diff: the retention time difference between the features

In [ ]:
conn <- find_connections(data = X, features = P, corr_thresh = 0.85, rt_window = 1/60, name_col = "Name", mz_col = "mz", rt_col = "rt")

The next part is where the actual ~~magic~~ science happens. The algorithm first constructs a network of the connected features, then starts stripping down the connected components of the network until we are left with densely connected clusters. The threshold for the relative degree can be set by the user. For example, using a degree threshold of 0.8 means that all the features in a cluster need to be connected with 80% of the other features of the cluster.

In [ ]:
clusters <- find_clusters(conn)

The resulting list has one element per cluster, each containing a character vector of feature names and an igraph object of the cluster graph. The graph object is useful for plotting

Now that we know which features are clustered together we can create modified versions of the original intensity and feature information data frames. 

In [ ]:
pulled <- pull_features(clusters, data= X, features = P, name_col = "Name")
cdata <- pulled$cdata
cpeaks <- pulled$cfeatures

The follwowing function will then save the modified data frames as separate sheet to the given Excel spreadsheet file.

In [ ]:
multisheet_xlsx(dfs = pulled, filename = "HILIC_NEG_output2.xlsx")

#### Visualizations

Let's pick a cluster and take a closer look at it:

In [ ]:
cluster <- clusters[[2]]
cluster$features

The cluster seems to have X features. We can also visualize the relationship between the fetures:

In [ ]:
plot_graph(features = P, cluster, name_col = "Name",
             mz_col = "mz", rt_col = "rt")

In [ ]:
plot_features(P, cluster, name_col = "Name", mz_col = "mz", rt_col = "rt", rt_window = 1/60)

In [ ]:
plot_heatmaps(data = X, features = P, name_col = "Name", mz_col = "mz", rt_col = "rt")